In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from pathlib import Path
from typing import List, Tuple, Dict

import cv2
import dataclass_array as dca
import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import pandas as pd
from PIL import Image
import scipy.io
from sklearn import linear_model
import transforms3d as t3d
import visu3d as v3d

from barrelnet.dust3r_utils import read_dust3r
from barrelnet.utils import cmapvals, segment_pc_from_mask

In [ ]:
H, W = (224, 512)
image_dir = Path("data/barrelddt1")
# assumption: dust3r data sorted by image name alphabetically
imgpaths = sorted(image_dir.glob("*.jpg"))

output_dir = Path(f"results/{image_dir.name}-reconstr")
dustpath = output_dir / "dust3r_out.pth"

pts_all, pts_each, v3dcams = read_dust3r(dustpath, W, H)

img_navs = pd.read_csv(image_dir / "frame-time-nav.csv")
img_navs["timestamp"] = pd.to_datetime(img_navs["timestamp"])

imgid2mask = {}
for idx, img in enumerate(imgpaths):
    imgname = Path(img.name).stem
    maskpath = output_dir / f"masks/{imgname}_mask_1.png"
    if maskpath.exists():
        # imgid2mask[idx] = cv2.imread(str(maskpath), cv2.IMREAD_GRAYSCALE)
        maskpil = Image.open(maskpath).convert("1").resize((W, H), Image.Resampling.NEAREST)
        imgid2mask[idx] = np.asarray(maskpil)

pc = v3d.Point3d(p=pts_each[0])


v3d.make_fig([
    pc,
    v3dcams
])

In [ ]:
# camrender = v3dcams[0].render(pc)
# fig, ax = plt.subplots()
# ax.imshow(camrender)
# plt.show()

In [ ]:
npts = pc.shape[0]
idxs = np.arange(npts)
barrelscores = np.zeros(npts)
for i, mask in imgid2mask.items():
    v3dcam = v3dcams[i]
    barrelidxs = segment_pc_from_mask(pc, mask, v3dcam)
    barrelscores[barrelidxs] += 1
barrelyes = barrelscores > len(imgid2mask) / 3
segcols = np.zeros_like(pc.p, dtype=np.uint8)
segcols[barrelyes] = [50, 222, 100]
segcols[~barrelyes] = [255, 0, 0]
segpc = v3d.Point3d(p=pc.p, rgb=segcols)
v3d.make_fig(segpc)